Use KL Divergence loss on Knowledge Distillation Task. You can use any teacher and student model (prefer small models). You need to show that it works, and update README.md with proper logs


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

try:
    from torchsummary import summary
except ModuleNotFoundError:
    !pip install torchsummary
    from torchsummary import summary

from torchvision import datasets, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import torchvision

import os
import time
import math

In [2]:
train_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                      #  transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                       transforms.RandomAffine(degrees=10, shear = 10),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

train = datasets.CIFAR10(root = './data', train=True, download=True, transform=train_transforms)
test = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transforms)

# Do we have CUDA drivers for us?
cuda = torch.cuda.is_available()
print ("Cuda Available?", cuda)

dataloader_args = dict(shuffle=True, batch_size=2048, num_workers=2, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# Dataloaders
train_loader = torch.utils.data.DataLoader(dataset=train, **dataloader_args)
test_loader = torch.utils.data.DataLoader(dataset=test, **dataloader_args)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170M/170M [00:05<00:00, 30.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Cuda Available? True


In [3]:
class TeacherModel(nn.Module):
    def __init__(self):
        super(TeacherModel, self).__init__()
        self.conv01 = nn.Conv2d(3, 16, 3, bias=False, padding=1)
        self.batch01 = nn.BatchNorm2d(num_features=16)

        # ---- Lets take a skip connection
        self.skip_conv1 = nn.Conv2d(16, 16, 3, padding=0, dilation=2)

        self.conv02 = nn.Conv2d(16, 16, 3, bias=False,padding=1)
        self.batch02 = nn.BatchNorm2d(num_features=16)
        self.conv03 = nn.Conv2d(16, 16, 3, bias=False,padding=1)
        self.batch03 = nn.BatchNorm2d(num_features=16)
        self.conv04 = nn.Conv2d(16, 16, 3, bias=False,padding=1)
        self.batch04 = nn.BatchNorm2d(num_features=16)
        self.pool01 = nn.MaxPool2d(2, 2)                                #O=16
        self.conv05 = nn.Conv2d(16, 16, 1, bias=False)

        self.conv11 = nn.Conv2d(16, 64, 3, bias=False, padding=1)
        self.batch11 = nn.BatchNorm2d(num_features=64)
        self.conv12 = nn.Conv2d(64, 64, 3, bias=False, padding=1)
        self.batch12 = nn.BatchNorm2d(num_features=64)
        self.conv13 = nn.Conv2d(64, 64, 3, bias=False, padding=1)
        self.batch13 = nn.BatchNorm2d(num_features=64)
        self.conv14 = nn.Conv2d(64, 64, 3, bias=False, padding=1)
        self.batch14 = nn.BatchNorm2d(num_features=64)
        self.pool11 = nn.MaxPool2d(2, 2)                                #O=8
        self.conv15 = nn.Conv2d(64, 64, 1, bias=False)

        self.conv21 = nn.Conv2d(64, 128, 3, bias=False, padding=1)
        self.batch21 = nn.BatchNorm2d(num_features=128)
        self.conv22 = nn.Conv2d(128, 128, 3, bias=False, padding=1)
        self.batch22 = nn.BatchNorm2d(num_features=128)
        self.conv23 = nn.Conv2d(128,128, 3, bias=False, padding=1)
        self.batch23 = nn.BatchNorm2d(num_features=128)
        self.conv24 = nn.Conv2d(128, 128, 3, bias=False, padding=1)
        self.batch24 = nn.BatchNorm2d(num_features=128)
        self.pool21 = nn.MaxPool2d(2, 2)                                #O=4
        self.conv25 = nn.Conv2d(128, 128, 1, bias=False)

        self.conv31 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, groups=128, bias = False, padding = 1)
        self.convPV1= nn.Conv2d(in_channels=128, out_channels=128, kernel_size=1, bias = False, padding = 0)
        self.batch31 = nn.BatchNorm2d(num_features=128)
        self.conv32 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, groups=128, bias = False, padding = 1)
        self.convPV2= nn.Conv2d(in_channels=128, out_channels=256, kernel_size=1, bias = False, padding = 0)
        self.batch32 = nn.BatchNorm2d(num_features=256)


        self.avg_pool = nn.AvgPool2d(kernel_size=4)
        self.convx3 = nn.Conv2d(256, 10, 1, bias=False, padding=0)

    def forward(self, x):
        x = self.batch01(F.relu(self.conv01(x)))

        # ---- Lets take a skip connection
        skip_channels = self.skip_conv1(self.skip_conv1(self.skip_conv1(self.skip_conv1(x))))

        x = self.batch02(F.relu(self.conv02(x)))
        x = self.batch03(F.relu(self.conv03(x)))
        x = self.batch04(F.relu(self.conv04(x)))
        x = self.pool01(x)
        x = self.conv05(x)
        # ----------------------------------------------------------

        # ---- Lets add the skip connection here
        x = skip_channels + x

        x = self.batch11(F.relu(self.conv11(x)))
        x = self.batch12(F.relu(self.conv12(x)))
        x = self.batch13(F.relu(self.conv13(x)))
        x = self.batch14(F.relu(self.conv14(x)))
        x = self.pool11(x)
        x = self.conv15(x)
        # ----------------------------------------------------------

        x = self.batch21(F.relu(self.conv21(x)))
        x = self.batch22(F.relu(self.conv22(x)))
        x = self.batch23(F.relu(self.conv23(x)))
        x = self.batch24(F.relu(self.conv24(x)))
        x = self.pool21(x)
        x = self.conv25(x)
        # ----------------------------------------------------------

        x = self.batch31(F.relu(self.convPV1(F.relu(self.conv31(x)))))
        x = self.batch32(F.relu(self.convPV2(F.relu(self.conv32(x)))))


        x = self.avg_pool(x)
        x = self.convx3(x)
        x = x.view(-1, 10)                           # Don't want 10x1x1..
        return F.log_softmax(x, dim=1)  # Added dim=1 parameter)

In [4]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []
time_taken = []

class EarlyStopping:
    def __init__(self, tolerance=5, min_delta=0.01):
        self.tolerance = tolerance
        self.min_delta = min_delta
        self.prev_loss = None  # Initialize as None
        self.counter = 0

    def __call__(self, train_loss):
        if self.prev_loss is None:  # First iteration
            self.prev_loss = train_loss
            return False  # Continue training

        if (abs(train_loss - self.prev_loss)) < self.min_delta:
            print(f'---------- prev = {self.prev_loss} current = {train_loss} ---------')
            self.counter += 1
        else:
            self.counter = 0  # Reset counter if loss improves

        self.prev_loss = train_loss

        return self.counter >= self.tolerance  # Return True if stopping criteria met



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)

    correct = 0
    processed = 0
    epoch_loss = 0
    time_taken.clear()

    for batch_idx, (data, target) in enumerate(pbar):
        t0 = time.time()

        data, target = data.to(device), target.to(device)

        # Don't want history of gradients
        optimizer.zero_grad()

        y_predict = model(data)

        # Calculate loss
        loss = F.nll_loss(y_predict, target)
        epoch_loss += loss.item()

        # Backpropagate error
        loss.backward()

        # Take an optimizer step
        optimizer.step()

        torch.cuda.synchronize()
        t1 = time.time()

        time_taken.append((t1 - t0))

        pred = y_predict.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc=f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100 * correct / processed:0.2f}')
        train_acc.append(100 * correct / processed)

    avg_train_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    return avg_train_loss


def test(model, device, test_loader):
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)

            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))
    return test_loss

In [5]:
# Initialize model, optimizer, and early stopping
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (f'Device Using = {device}')
model = TeacherModel().to(device)
summary(model, input_size=(3, 32, 32))
criteria = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
early_stopping = EarlyStopping(tolerance=5, min_delta=0.02)

EPOCHS = 100
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    avg_train_loss = train(model, device, train_loader, optimizer, epoch)
    print(f" --> EPOCH: {epoch}, Avg Training Loss: {avg_train_loss:.4f}, Avg Time Taken = {(sum(time_taken) / len(time_taken)) * 1000:.2f}ms")
    val_loss = test(model, device, test_loader)

    # Check for early stopping
    if early_stopping(avg_train_loss):
        try:
            # Ensure the directory exists
            save_dir = '/content/drive/MyDrive/EPAi_V5'
            os.makedirs(save_dir, exist_ok=True)
            PATH = os.path.join(save_dir, f'model_heavy_acc_{int(train_acc[-1]):d}.pth')
        except OSError:
            # Fallback to current directory if Drive is unavailable
            PATH = f'./model_heavy_acc_{int(train_acc[-1]):d}.pth'

        # Save the model weights
        torch.save(model.state_dict(), PATH)
        print(f"Model saved at: {PATH}")
        print("Early stopping triggered!")
        break

Device Using = cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
            Conv2d-3           [-1, 16, 28, 28]           2,320
            Conv2d-4           [-1, 16, 24, 24]           2,320
            Conv2d-5           [-1, 16, 20, 20]           2,320
            Conv2d-6           [-1, 16, 16, 16]           2,320
            Conv2d-7           [-1, 16, 32, 32]           2,304
       BatchNorm2d-8           [-1, 16, 32, 32]              32
            Conv2d-9           [-1, 16, 32, 32]           2,304
      BatchNorm2d-10           [-1, 16, 32, 32]              32
           Conv2d-11           [-1, 16, 32, 32]           2,304
      BatchNorm2d-12           [-1, 16, 32, 32]              32
        MaxPool2d-13           [-1, 16, 16, 16]               0
           Conv2d-1

Loss=1.7319616079330444 Batch_id=24 Accuracy=25.04: 100%|██████████| 25/25 [00:25<00:00,  1.03s/it]

 --> EPOCH: 0, Avg Training Loss: 1.9959, Avg Time Taken = 444.17ms



Test set: Average loss: 2.3616, Accuracy: 1000/10000 (10.00%)

Epoch 2/100


Loss=1.485198974609375 Batch_id=24 Accuracy=42.36: 100%|██████████| 25/25 [00:17<00:00,  1.42it/s]

...

 --> EPOCH: 30, Avg Training Loss: 0.2412, Avg Time Taken = 428.61ms



Test set: Average loss: 0.7455, Accuracy: 7845/10000 (78.45%)

---------- prev = 0.24509883403778077 current = 0.2411676061153412 ---------
Epoch 32/100


Loss=0.21317756175994873 Batch_id=24 Accuracy=92.10: 100%|██████████| 25/25 [00:17<00:00,  1.39it/s]

 --> EPOCH: 31, Avg Training Loss: 0.2304, Avg Time Taken = 429.03ms



Test set: Average loss: 0.7414, Accuracy: 7891/10000 (78.91%)

---------- prev = 0.2411676061153412 current = 0.2303864675760269 ---------
Model saved at: /content/drive/MyDrive/EPAi_V5/model_heavy_acc_92.pth
Early stopping triggered!


In [6]:
model.load_state_dict(torch.load('/content/drive/MyDrive/EPAi_V5/model_heavy_acc_92.pth', weights_only=True))
test(model, device, test_loader)


Test set: Average loss: 0.7414, Accuracy: 7891/10000 (78.91%)



0.7414170166015625

In [7]:
class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.conv01 = nn.Conv2d(3, 16, 3, bias=False, padding=1)
        self.batch01 = nn.BatchNorm2d(num_features=16)

        # ---- Lets take a skip connection
        self.skip_conv1 = nn.Conv2d(16, 16, 3, padding=0, dilation=2)

        self.conv02 = nn.Conv2d(16, 16, 3, bias=False,padding=1)
        self.batch02 = nn.BatchNorm2d(num_features=16)
        self.conv03 = nn.Conv2d(16, 16, 3, bias=False,padding=1)
        self.batch03 = nn.BatchNorm2d(num_features=16)
        self.conv04 = nn.Conv2d(16, 16, 3, bias=False,padding=1)
        self.batch04 = nn.BatchNorm2d(num_features=16)
        self.pool01 = nn.MaxPool2d(2, 2)                                #O=16
        self.conv05 = nn.Conv2d(16, 16, 1, bias=False)

        self.conv11 = nn.Conv2d(16, 32, 3, bias=False, padding=1)
        self.batch11 = nn.BatchNorm2d(num_features=32)
        self.conv12 = nn.Conv2d(32, 32, 3, bias=False, padding=1)
        self.batch12 = nn.BatchNorm2d(num_features=32)
        self.conv13 = nn.Conv2d(32, 32, 3, bias=False, padding=1)
        self.batch13 = nn.BatchNorm2d(num_features=32)
        self.conv14 = nn.Conv2d(32, 32, 3, bias=False, padding=1)
        self.batch14 = nn.BatchNorm2d(num_features=32)
        self.pool11 = nn.MaxPool2d(2, 2)                                #O=8
        self.conv15 = nn.Conv2d(32, 32, 1, bias=False)

        self.conv21 = nn.Conv2d(32, 64, 3, bias=False, padding=1)
        self.batch21 = nn.BatchNorm2d(num_features=64)
        self.conv22 = nn.Conv2d(64, 64, 3, bias=False, padding=1)
        self.batch22 = nn.BatchNorm2d(num_features=64)
        self.conv23 = nn.Conv2d(64, 64, 3, bias=False, padding=1)
        self.batch23 = nn.BatchNorm2d(num_features=64)
        self.conv24 = nn.Conv2d(64, 64, 3, bias=False, padding=1)
        self.batch24 = nn.BatchNorm2d(num_features=64)
        self.pool21 = nn.MaxPool2d(2, 2)                                #O=4
        self.conv25 = nn.Conv2d(64, 64, 1, bias=False)

        self.conv31 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, groups=64, bias = False, padding = 1)
        self.convPV1= nn.Conv2d(in_channels=64, out_channels=128, kernel_size=1, bias = False, padding = 0)
        self.batch31 = nn.BatchNorm2d(num_features=128)
        self.conv32 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, groups=128, bias = False, padding = 1)
        self.convPV2= nn.Conv2d(in_channels=128, out_channels=256, kernel_size=1, bias = False, padding = 0)
        self.batch32 = nn.BatchNorm2d(num_features=256)


        self.avg_pool = nn.AvgPool2d(kernel_size=4)
        self.convx3 = nn.Conv2d(256, 10, 1, bias=False, padding=0)

    def forward(self, x):
        x = self.batch01(F.relu(self.conv01(x)))

        # ---- Lets take a skip connection
        skip_channels = self.skip_conv1(self.skip_conv1(self.skip_conv1(self.skip_conv1(x))))

        x = self.batch02(F.relu(self.conv02(x)))
        x = self.batch03(F.relu(self.conv03(x)))
        x = self.batch04(F.relu(self.conv04(x)))
        x = self.pool01(x)
        x = self.conv05(x)
        # ----------------------------------------------------------

        # ---- Lets add the skip connection here
        x = skip_channels + x

        x = self.batch11(F.relu(self.conv11(x)))
        x = self.batch12(F.relu(self.conv12(x)))
        x = self.batch13(F.relu(self.conv13(x)))
        x = self.batch14(F.relu(self.conv14(x)))
        x = self.pool11(x)
        x = self.conv15(x)
        # ----------------------------------------------------------

        x = self.batch21(F.relu(self.conv21(x)))
        x = self.batch22(F.relu(self.conv22(x)))
        x = self.batch23(F.relu(self.conv23(x)))
        x = self.batch24(F.relu(self.conv24(x)))
        x = self.pool21(x)
        x = self.conv25(x)
        # ----------------------------------------------------------

        x = self.batch31(F.relu(self.convPV1(F.relu(self.conv31(x)))))
        x = self.batch32(F.relu(self.convPV2(F.relu(self.conv32(x)))))


        x = self.avg_pool(x)
        x = self.convx3(x)
        x = x.view(-1, 10)                           # Don't want 10x1x1..
        return F.log_softmax(x, dim=1)  # Added dim=1 parameter)

In [8]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []
time_taken = []

class EarlyStopping:
    def __init__(self, tolerance=5, min_delta=0.01):
        self.tolerance = tolerance
        self.min_delta = min_delta
        self.prev_loss = None  # Initialize as None
        self.counter = 0

    def __call__(self, train_loss):
        if self.prev_loss is None:  # First iteration
            self.prev_loss = train_loss
            return False  # Continue training

        if (abs(train_loss - self.prev_loss)) < self.min_delta:
            print(f'---------- prev = {self.prev_loss} current = {train_loss} ---------')
            self.counter += 1
        else:
            self.counter = 0  # Reset counter if loss improves

        self.prev_loss = train_loss

        return self.counter >= self.tolerance  # Return True if stopping criteria met



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)

    correct = 0
    processed = 0
    epoch_loss = 0
    time_taken.clear()

    for batch_idx, (data, target) in enumerate(pbar):
        t0 = time.time()

        data, target = data.to(device), target.to(device)

        # Don't want history of gradients
        optimizer.zero_grad()

        y_predict = model(data)

        # Calculate loss
        loss = F.nll_loss(y_predict, target)
        epoch_loss += loss.item()

        # Backpropagate error
        loss.backward()

        # Take an optimizer step
        optimizer.step()

        torch.cuda.synchronize()
        t1 = time.time()

        time_taken.append((t1 - t0))

        pred = y_predict.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc=f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100 * correct / processed:0.2f}')
        train_acc.append(100 * correct / processed)

    avg_train_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    return avg_train_loss


def test(model, device, test_loader):
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)

            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))
    return test_loss

In [9]:
# Initialize model, optimizer, and early stopping
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (f'Device Using = {device}')
model = StudentModel().to(device)
summary(model, input_size=(3, 32, 32))
criteria = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
early_stopping = EarlyStopping(tolerance=5, min_delta=0.02)

EPOCHS = 100
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    avg_train_loss = train(model, device, train_loader, optimizer, epoch)
    print(f" --> EPOCH: {epoch}, Avg Training Loss: {avg_train_loss:.4f}, Avg Time Taken = {(sum(time_taken) / len(time_taken)) * 1000:.2f}ms")
    val_loss = test(model, device, test_loader)

    # Check for early stopping
    if early_stopping(avg_train_loss):
        try:
            # Ensure the directory exists
            save_dir = '/content/drive/MyDrive/EPAi_V5'
            os.makedirs(save_dir, exist_ok=True)
            PATH = os.path.join(save_dir, f'model_small_acc_{int(train_acc[-1]):d}.pth')
        except OSError:
            # Fallback to current directory if Drive is unavailable
            PATH = f'./model_small_acc_{int(train_acc[-1]):d}.pth'

        # Save the model weights
        torch.save(model.state_dict(), PATH)
        print(f"Model saved at: {PATH}")
        print("Early stopping triggered!")
        break

Device Using = cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
            Conv2d-3           [-1, 16, 28, 28]           2,320
            Conv2d-4           [-1, 16, 24, 24]           2,320
            Conv2d-5           [-1, 16, 20, 20]           2,320
            Conv2d-6           [-1, 16, 16, 16]           2,320
            Conv2d-7           [-1, 16, 32, 32]           2,304
       BatchNorm2d-8           [-1, 16, 32, 32]              32
            Conv2d-9           [-1, 16, 32, 32]           2,304
      BatchNorm2d-10           [-1, 16, 32, 32]              32
           Conv2d-11           [-1, 16, 32, 32]           2,304
      BatchNorm2d-12           [-1, 16, 32, 32]              32
        MaxPool2d-13           [-1, 16, 16, 16]               0
           Conv2d-1

Loss=1.839409351348877 Batch_id=24 Accuracy=19.72: 100%|██████████| 25/25 [00:16<00:00,  1.55it/s]

 --> EPOCH: 0, Avg Training Loss: 2.1232, Avg Time Taken = 342.92ms



Test set: Average loss: 2.2944, Accuracy: 998/10000 (9.98%)

Epoch 2/100


Loss=1.6231263875961304 Batch_id=24 Accuracy=36.33: 100%|██████████| 25/25 [00:17<00:00,  1.45it/s]

...

 --> EPOCH: 24, Avg Training Loss: 0.5336, Avg Time Taken = 336.70ms



Test set: Average loss: 0.7186, Accuracy: 7558/10000 (75.58%)

---------- prev = 0.5526035451889038 current = 0.5336136102676392 ---------
Epoch 26/100


Loss=0.5731205344200134 Batch_id=24 Accuracy=81.76: 100%|██████████| 25/25 [00:16<00:00,  1.53it/s]

 --> EPOCH: 25, Avg Training Loss: 0.5237, Avg Time Taken = 334.28ms



Test set: Average loss: 0.7493, Accuracy: 7525/10000 (75.25%)

---------- prev = 0.5336136102676392 current = 0.5237028408050537 ---------
Model saved at: /content/drive/MyDrive/EPAi_V5/model_small_acc_81.pth
Early stopping triggered!


In [10]:
model.load_state_dict(torch.load('/content/drive/MyDrive/EPAi_V5/model_small_acc_81.pth', weights_only=True))
test(model, device, test_loader)


Test set: Average loss: 0.7493, Accuracy: 7525/10000 (75.25%)



0.74932431640625

# Distillation Process

In [11]:
class EarlyStopping:
    def __init__(self, tolerance=5, min_delta=0.01):
        self.tolerance = tolerance
        self.min_delta = min_delta
        self.prev_loss = None  # Initialize as None
        self.counter = 0

    def __call__(self, train_loss):
        if self.prev_loss is None:  # First iteration
            self.prev_loss = train_loss
            return False  # Continue training

        if (abs(train_loss - self.prev_loss)) < self.min_delta:
            print(f'---------- prev = {self.prev_loss} current = {train_loss} ---------')
            self.counter += 1
        else:
            self.counter = 0  # Reset counter if loss improves

        self.prev_loss = train_loss

        return self.counter >= self.tolerance  # Return True if stopping criteria met

train_losses = []
test_losses = []
train_acc = []
test_acc = []
time_taken = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device = {device}')

# Prepare teacher model
teacher = TeacherModel().to(device)
teacher.load_state_dict(torch.load('/content/drive/MyDrive/EPAi_V5/model_heavy_acc_92.pth', weights_only=True))
teacher.eval()

# Prepare student model
student = StudentModel().to(device)
student.load_state_dict(torch.load('/content/drive/MyDrive/EPAi_V5/model_small_acc_81.pth', weights_only=True))

print("=============================================")
print("Student model accuracy before training ")
student.eval()
test(student, device, test_loader)
print("=============================================")

# Loss functions
hard_loss = nn.CrossEntropyLoss() #Hard label loss
soft_loss = nn.KLDivLoss(reduction="batchmean")  # Distillation loss

# Temperature and alpha
T = 5.0  # Temperature
alpha = 0.5  # Weight for distillation loss

# Optimizer
optimizer = optim.Adam(student.parameters(), lr=0.001)

# Training loop
EPOCHS = 100
for epoch in range(EPOCHS):

    correct = 0
    processed = 0
    epoch_loss = 0
    pbar = tqdm(train_loader)
    student.train()
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        # Teacher predictions (soft labels)
        with torch.no_grad():
            teacher_logits = teacher(data) / T

        teacher_probs = torch.softmax(teacher_logits, dim=1)

        # Student predictions
        student_logits = student(data)
        student_probs = torch.log_softmax(student_logits / T, dim=1)

        # Compute losses
        loss_soft = soft_loss(student_probs, teacher_probs) * (T ** 2)  # Scale by T^2
        loss_hard = hard_loss(student_logits, target)
        loss = alpha * loss_hard + (1 - alpha) * loss_soft

        epoch_loss += loss.item()

        # Backpropagation
        loss.backward()
        optimizer.step()

        pred = student_logits.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc=f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100 * correct / processed:0.2f}')

    avg_train_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    train_acc.append(100 * correct / processed) # Append to train_acc here

    # Check for early stopping
    if early_stopping(avg_train_loss):
        try:
            # Ensure the directory exists
            save_dir = '/content/drive/MyDrive/EPAi_V5'
            os.makedirs(save_dir, exist_ok=True)
            PATH = os.path.join(save_dir, f'model_distil_acc_{int(train_acc[-1]):d}.pth')
        except OSError:
            # Fallback to current directory if Drive is unavailable
            PATH = f'./model_small_acc_{int(train_acc[-1]):d}.pth'

        # Save the model weights
        torch.save(model.state_dict(), PATH)
        print(f"Model saved at: {PATH}")
        print("Early stopping triggered!")
        break

    student.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            output = student(data)

            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



Using device = cuda
Student model accuracy before training 

Test set: Average loss: 0.7493, Accuracy: 7525/10000 (75.25%)



Loss=1.246791958808899 Batch_id=24 Accuracy=76.08: 100%|██████████| 25/25 [00:17<00:00,  1.41it/s]



Test set: Average loss: 0.8613, Accuracy: 7480/10000 (74.80%)



Loss=1.036960482597351 Batch_id=24 Accuracy=79.96: 100%|██████████| 25/25 [00:17<00:00,  1.42it/s]



Test set: Average loss: 0.7897, Accuracy: 7633/10000 (76.33%)



...

Loss=0.6941761374473572 Batch_id=24 Accuracy=87.25: 100%|██████████| 25/25 [00:17<00:00,  1.43it/s]

---------- prev = 0.6972872185707092 current = 0.6909924077987671 ---------



Test set: Average loss: 0.6660, Accuracy: 7970/10000 (79.70%)



Loss=0.7098481059074402 Batch_id=24 Accuracy=87.79: 100%|██████████| 25/25 [00:18<00:00,  1.36it/s]

---------- prev = 0.6909924077987671 current = 0.6800520181655884 ---------
Model saved at: /content/drive/MyDrive/EPAi_V5/model_distil_acc_87.pth
Early stopping triggered!
